# **ETAPA 2 - IMPORTAÇÃO DAS BASES DE DADOS**

<br>

---

<br>

Agora será feito a importação dos dados para a camada especificada do Data LakeHouse que já está pronta.

*`Esse é um modelo modular, complete as informações necessárias nos trechos que estão destacados em vermelho assim como esse, seguindo o padrão snake_case.`*

<br> 

***AVISO**: Esse Notebook foi feito com base na estrutura do Databricks Free Edition, que utiliza catálogos.*

<br><br>

---

<br>

### Parte 1 - **Importação das Bibliotecas Necessárias**

In [0]:
# import pyspark 
import urllib
import os
import datetime
import shutil

import importlib
import funcoes_log
importlib.reload(funcoes_log)
from funcoes_log import create_pre_log, finalize_log

<br>

---

<br>

### Parte 2 - **Importação dos Arquivos para o Data Lakehouse**

`Insira nas variáveis:` <br>
`--> nome_datalakehouse --> nome do Data Lakehouse destino` <br>
`--> nome_camada --> o nome da camada destino dos dados totalmente brutos` <br>
`--> nome_volume_atual --> nome do volume onde ficaram os dados atualizados` <br>
`--> nome_volume_historico --> nome do volume onde ficaram o histórico dos dados` <br>

In [0]:
nome_datalakehouse = "dataexperts"
nome_camada = "landing_zone"

nome_volume_atual = "vendas_atual"
nome_volume_historico = "vendas_historico"

O código a seguir cria os volumes se eles ainda não existirem:

In [0]:
spark.sql(f"""
CREATE VOLUME IF NOT EXISTS {nome_datalakehouse}.{nome_camada}.{nome_volume_atual}
""")

spark.sql(f"""
CREATE VOLUME IF NOT EXISTS {nome_datalakehouse}.{nome_camada}.{nome_volume_historico}
""")

`Insira nos vetores todos os nomes dos arquivos que deseja importar e a origem deles, seguindo a mesma ordem:`

In [0]:
arquivos = [
    "categoria_produto.csv",
    "cliente.csv",
    "data.csv",
    "localidade.csv",
    "produto.csv",
    "vendas_part1.csv",
    "vendas_part2.csv",
    "vendas_part3.csv",
    "vendas_part4.csv"
]

origem_arquivos = [
    "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/",
    "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/",
    "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/",
    "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/",
    "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/",
    "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/",
    "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/",
    "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/",
    "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/",
]

O código a seguir verifica se o arquivo já existe no Data Lakehouse:
- Se sim, apenas pula para o próximo download (se ainda exitir arquivos na fila em espera)
- Se não, tenta baixar o arquivo da origem.
Logo após, caso o download dê certo ou não, passa para a próximo download.

In [0]:
destino_atual = f"/Volumes/{nome_datalakehouse}/{nome_camada}/{nome_volume_atual}/"
destino_historico = f"/Volumes/{nome_datalakehouse}/{nome_camada}/{nome_volume_historico}/"

for arquivo, origem in zip(arquivos, origem_arquivos):

    caminho_atual = os.path.join(destino_atual, arquivo)

    TABELA_LOG = "dataexperts.audit.logs"

    log = create_pre_log(
        camada="landing_zone",
        tabela_log=TABELA_LOG,
        job_nome="landing_zone_download_csv",
        acao="DOWNLOAD",
        nome_objeto=arquivo,
        fonte="GITHUB",
        path_origem=origem + arquivo,
        path_destino=caminho_atual
    )

    try:
        if os.path.exists(caminho_atual):
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            nome_historico = arquivo.replace(".csv", f"_{timestamp}.csv")
            caminho_historico = os.path.join(destino_historico, nome_historico)
            shutil.move(caminho_atual, caminho_historico)

        urllib.request.urlretrieve(origem + arquivo, caminho_atual)
        print(log.keys())

        finalize_log(
            spark,
            log,
            status="SUCCESS",
            registros_gravados=1
        )

    except Exception as e:
        finalize_log(
            spark,
            log,
            status="ERROR",
            mensagem_erro=str(e)
        )
        raise


In [0]:
%sql
DESCRIBE TABLE dataexperts.logs.bronze_historico;


<br>

---

<br>

### **Resultado Final**

O código a seguir mostra todos os arquivos mais recentes salvos:

In [0]:
# Código está comentado por ser apenas uma demonstração. Para visualizar o resultado, retire o identificador de comentário e execute novamente

display(dbutils.fs.ls(destino_atual))


O código a seguir mostra o histórico de todas as versões dos arquivos salvos:

In [0]:
# Código está comentado por ser apenas uma demonstração. Para visualizar o resultado, retire o identificador de comentário e execute novamente

display(dbutils.fs.ls(destino_historico))

O código a seguir mostra um arquivo como exemplo:

In [0]:
# Código está comentado por ser apenas uma demonstração. Para visualizar o resultado, retire o identificador de comentário e execute novamente

arquivo_exemplo = f"{destino_atual}/{arquivos[0]}"

dataframe_arquivo_exemplo = spark.read.csv(arquivo_exemplo, header=True)

display(dataframe_arquivo_exemplo)